In [ ]:
import json 
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
import matplotlib.pyplot as plt

files = os.listdir('../Data/Analysed')

datas = []
players = {}
all_obs = [] 
games = []
# playername, rating, difference, time 
for file in tqdm(files):
    with open('../Data/Analysed/' + file) as f:
        data = json.load(f)
    
    white_moves_time, black_moves_time = data["game"]["moveTimestamps"].split(",")[:-1][::2], data["game"]["moveTimestamps"].split(",")[:-1][1::2]
    white_moves_time, black_moves_time = [int(x)/10 for x in white_moves_time], [int(x)/10 for x in black_moves_time]
    time_diffs_white, time_diffs_black = [0] + list(np.diff(white_moves_time)), [0] + list(np.diff(black_moves_time))
    
    user_black = data["players"]["top"]["username"]
    user_white = data["players"]["bottom"]["username"]
    rating_black = data["players"]["top"]["rating"]
    rating_white = data["players"]["bottom"]["rating"]

    white_loss = [x["difference"] for x in data["evaluations"] if x["player"] == "white"]
    black_loss = [x["difference"] for x in data["evaluations"] if x["player"] == "black"]
    white_loss_after_10 = np.median(white_loss[10:])
    black_loss_after_10 = np.median(black_loss[10:])
    white_loss = np.median(white_loss)
    black_loss = np.median(black_loss)
    white_perc_best_move = np.mean([x["Ranking Real Move"] == 1 for x in data["evaluations"] if x["player"] == "white"])
    black_perc_best_move = np.mean([x["Ranking Real Move"] == 1 for x in data["evaluations"] if x["player"] == "black"])
    white_perc_very_good_move = np.mean([x["Ranking Real Move"] <= 3 for x in data["evaluations"] if x["player"] == "white"])
    black_perc_very_good_move = np.mean([x["Ranking Real Move"] <= 3 for x in data["evaluations"] if x["player"] == "black"])

    white_perc_best_move_after_10 = np.mean([x["Ranking Real Move"] == 1 for x in data["evaluations"][10:] if x["player"] == "white"])
    black_perc_best_move_after_10 = np.mean([x["Ranking Real Move"] == 1 for x in data["evaluations"][10:] if x["player"] == "black"])
    # Ranking Real Move (if -1 => 10)
    avg_rank_white = np.mean([x["Ranking Real Move"] if x["Ranking Real Move"] > 0 else 10 for x in data["evaluations"] if x["player"] == "white"])
    avg_rank_black = np.mean([x["Ranking Real Move"] if x["Ranking Real Move"] > 0 else 10 for x in data["evaluations"] if x["player"] == "black"])
    ranks_white = [x["Ranking Real Move"] if x["Ranking Real Move"] > 0 else 10 for x in data["evaluations"] if x["player"] == "white"]
    ranks_black = [x["Ranking Real Move"] if x["Ranking Real Move"] > 0 else 10 for x in data["evaluations"] if x["player"] == "black"]

    if user_black not in players:
        players[user_black] = {"games": 0, "loss": [], "loss_after_10": [], "perc_best_move": [], "perc_best_move_after_10": [], "rating": [], "avg_rank": [], "ranks": [], "very_good_move": []}
    if user_white not in players:
        players[user_white] = {"games": 0, "loss": [], "loss_after_10": [], "perc_best_move": [], "perc_best_move_after_10": [], "rating": [], "avg_rank": [], "ranks": [], "very_good_move": []}
    
    players[user_black]["games"] += 1
    players[user_white]["games"] += 1
    players[user_black]["loss"].append(black_loss)
    players[user_white]["loss"].append(white_loss)
    players[user_black]["loss_after_10"].append(black_loss_after_10)
    players[user_white]["loss_after_10"].append(white_loss_after_10)
    players[user_black]["perc_best_move"].append(black_perc_best_move)
    players[user_white]["perc_best_move"].append(white_perc_best_move)
    players[user_black]["perc_best_move_after_10"].append(black_perc_best_move_after_10)
    players[user_white]["perc_best_move_after_10"].append(white_perc_best_move_after_10)
    players[user_black]["rating"].append(rating_black)
    players[user_white]["rating"].append(rating_white)
    players[user_black]["avg_rank"].append(avg_rank_black)
    players[user_white]["avg_rank"].append(avg_rank_white)
    players[user_black]["ranks"].append(ranks_black)
    players[user_white]["ranks"].append(ranks_white)
    players[user_black]["very_good_move"].append(black_perc_very_good_move)
    players[user_white]["very_good_move"].append(white_perc_very_good_move)

    white_moves, black_moves = 0, 0
    for i in range(len(data["evaluations"])):
        white_moves += 1 if data["evaluations"][i]["player"] == "white" else 0
        black_moves += 1 if data["evaluations"][i]["player"] == "black" else 0
        all_obs.append({"player": user_black if i % 2 == 0 else user_white, 
                        "colour": 0 if i % 2 == 0 else 1,
                        "rating":  np.mean(players[user_black]["rating"]) if i % 2 == 0 else np.mean(players[user_white]["rating"]),
                        "difference": -np.abs(data["evaluations"][i]["Best Move Eval"] - data["evaluations"][i]["Real Move Eval"]),
                        "time": time_diffs_white[white_moves-1] if data["evaluations"][i]["player"] == "white" else time_diffs_black[black_moves-1],
                        "remaining_time": white_moves_time[white_moves-1] if data["evaluations"][i]["player"] == "white" else black_moves_time[black_moves-1],
                        "Ranking Real Move": data["evaluations"][i]["Ranking Real Move"],
                        "Move_Num" : white_moves if data["evaluations"][i]["player"] == "white" else black_moves,
                        "opponent": user_white if i % 2 == 0 else user_black,
                        "game_id": data["game"]["id"],
                        "oppenent_rating": np.mean(players[user_white]["rating"]) if i % 2 == 0 else np.mean(players[user_black]["rating"]),
                        "real_eval": data["evaluations"][i]["Real Move Eval"],
                        "best_eval": data["evaluations"][i]["Best Move Eval"]})

In [ ]:
games_sequences = {}
ratings = {}
for obs in all_obs:
    if f"""{obs["game_id"]}_{obs["player"]}""" not in games_sequences:
        games_sequences[f"""{obs["game_id"]}_{obs["player"]}"""] = []
    games_sequences[f"""{obs["game_id"]}_{obs["player"]}"""].append([obs["time"],obs["Ranking Real Move"], obs["real_eval"],  obs["best_eval"], obs["colour"], obs["difference"], obs["remaining_time"]])
#obs["difference"], 

    if f"""{obs["game_id"]}_{obs["player"]}""" not in ratings:
        ratings[f"""{obs["game_id"]}_{obs["player"]}"""] = obs["rating"]

print(len(games_sequences))
games_sequences = {k: v for k, v in games_sequences.items() if len(v) > 20}
print(len(games_sequences))

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.layers import Dropout, BatchNormalization
import keras
from keras.callbacks import Callback

# Functions for normalization and denormalization
def normalize(y):
    min_val = np.min(y)
    max_val = np.max(y)
    y_normalized = (y - min_val) / (max_val - min_val)
    return y_normalized, min_val, max_val

def denormalize(y_normalized, min_val, max_val):
    y_original = y_normalized * (max_val - min_val) + min_val
    return y_original

adjusted_sequences = {key: val[0:40] for key, val in games_sequences.items()}

# Ensure that the sequences in X and the ratings in y match up
X = []
y = []
for key in adjusted_sequences:
    X.append(adjusted_sequences[key])
    y.append(ratings[key])

# Padding the sequences to a fixed length of 30
X_padded = pad_sequences(X, maxlen=30, padding='post', dtype='float32')
# Normalize the ratings (y-values)
y_normalized, min_val, max_val = normalize(y)


In [ ]:
class CustomMetricsCallback(Callback):
    def __init__(self, X_data, y_data, min_val, max_val, n_splits=5):
        super().__init__()
        self.X_data = X_data
        self.y_data = y_data
        self.min_val = min_val
        self.max_val = max_val
        self.kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        self.split_gen = self.kf.split(self.X_data)

    def on_epoch_end(self, epoch, logs=None):
        try:
            _, test_index = next(self.split_gen)
        except StopIteration:
            self.split_gen = self.kf.split(self.X_data)
            _, test_index = next(self.split_gen)

        X_test, y_test_normalized = self.X_data[test_index], self.y_data[test_index]
        y_pred_normalized = self.model.predict(X_test)
        
        # Denormalize predictions and ground truth
        y_pred = denormalize(y_pred_normalized, self.min_val, self.max_val)
        y_test = denormalize(y_test_normalized, self.min_val, self.max_val)

        avg_pred = np.mean(y_pred)
        std_pred = np.std(y_pred)
        mae = np.mean(np.abs(y_pred - y_test))
        mse = np.mean((y_pred - y_test)**2)
        mad = np.mean(np.abs(y_pred - y_test))

        print("\nAvg Pred: {:.4f} | Std Pred: {:.4f} | MAE: {:.4f} | MSE: {:.4f} | MAD: {:.4f}".format(avg_pred, std_pred, mae, mse, mad))

def create_model():
    model = Sequential()
    model.add(LSTM(100, input_shape=(30, 7), return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

custom_metrics = CustomMetricsCallback(X_data=X_padded, y_data=y_normalized, min_val=min_val, max_val=max_val)

model = create_model()

model.fit(X_padded, y_normalized, 
          validation_data=(X_padded, y_normalized),  # Placeholder, not actually used for validation.
          epochs=100, 
          batch_size=16, 
          callbacks=[custom_metrics], 
          verbose=1)

# Write Model to Disk in ../Model/
model.save('../Model/elo_model.h5')

In [ ]:
model = keras.models.load_model('../Model/elo_model.h5')

In [ ]:
y_pred_normalized = model.predict(X_padded)
y_pred = denormalize(y_pred_normalized, min_val, max_val)
y_original = denormalize(y_normalized, min_val, max_val)

plt.scatter(y_original, y_pred, s = 10, edgecolor='black')
# Add 45 degree line
plt.plot([min(y_original), max(y_original)], [min(y_original), max(y_original)], 'k--', lw=3, color='red')
plt.xlabel("Ground Truth")
plt.ylabel("Predictions")
plt.title("Predictions vs Ground Truth. Elo Rating")
plt.show()